In [1]:
import os
from dotenv import load_dotenv

In [2]:
#loading my env file for this project which contains my secret key
load_dotenv()

True

In [3]:
# this is the part where we use our own secret key and check if the workflow is 
# working or not, we use the env and check it's validity

In [4]:
try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("Gemini setup complete")
except Exception as e:
    print(f"Authentication error {e}")

Gemini setup complete


In [5]:
# Now we are going to import Google's ADK components

In [6]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("ADK comp imported successfully")

ADK comp imported successfully


In [7]:
# we setup "Retry capabilities" - when a LLM fails to generate output sometime due to rate limit or temporary service unavailable
# we have these setup so failures can be handled by retrying the request with some backoff


In [8]:
retry_config = types.HttpRetryOptions(
    attempts = 5, # Max retry attempts
    exp_base = 7, # delay multiplier
    initial_delay = 1, # initial delay before first retry in seconds
    http_status_codes = [429, 500, 503, 504] # retry on these HTTP errors
)

In [9]:
# Section 2 Our first AI Agent with Google's ADK

## What is an AI agent?
### An AI agent is a tool which has the capability of interacting with real world parameters, such as API, external tools
### It has the ability to take an action, observe the result and based on the result take the next steps.

### An LLM is basically - user gives a prompt and it gives a textaul response 
### Prompt --> LLM --> Text

### An AI agent takes it one step further:
### Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer

# Section 2.1 - Defining an Agent

# These are the main properties we'll set:

<li>name and description: A simple name and description to identify our agent.</li>
<li>model: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".</li>
<li>instruction: The agent's guiding prompt. This tells the agent what its goal is and how to behave.</li>
<li>tools: A list of tools that the agent can use. To start, we'll give it the google_search tool, which lets it find up-to-date information online.</li>


In [10]:
try:
    root_agent = Agent(
        name = "helpful_assistant",
        model = Gemini(
            model = "gemini-2.5-flash-lite",
            retry_options = retry_config
        ),
        description = " A simple agent which can answer simple questions.",
        instruction = "You are a helpful assistant. Use google search for current info or if unsure.",
        tools=[google_search],
    )
    print("root_agent defined")
except Exception as e:
    print(f"Failed to define a root_agent with the desired properties {e}")

root_agent defined


# Section 2.2 - Running our Agent
To do this we need a runner, it is the central component of ADK that acts as an orchestrator. It manages teh conversations, sends our message to the agent, and handle it's responses.

In [11]:
# A) creating an inmemory_runner which will tell it to use our root_Agent

In [12]:
runner = InMemoryRunner(agent = root_agent)
print("runner created")

runner created


In [13]:
# B) Method to send our prompt and get an answer

In [14]:
response = await runner.run_debug(
    " What is Agent Development kit form Google? What language is the SDK available in? "
)


 ### Created new session: debug_session_id

User >  What is Agent Development kit form Google? What language is the SDK available in? 
helpful_assistant > The Google Agent Development Kit (ADK) is a flexible and modular framework designed to simplify the creation, deployment, and orchestration of AI agents. It aims to make agent development more akin to traditional software development, allowing for the creation of agentic architectures that can handle tasks ranging from simple to complex. ADK is model-agnostic and deployment-agnostic, meaning it can work with various AI models and be deployed in different environments. It also emphasizes interoperability, allowing agents built with ADK to communicate with agents created using other frameworks.

Key features of the ADK include:
*   **Code-First Development**: Developers define agent logic, tools, and orchestration directly in code, offering flexibility and testability.
*   **Modular Multi-Agent Systems**: ADK facilitates the compositi

# 2.4 How does it work?¶
The agent performed a Google Search to get the latest information about ADK, and it knew to use this tool because:

The agent inspects and is aware of which tools it has available to use.
The agent's instructions specify the use of the search tool to get current information or if it is unsure of an answer.
The best way to see the full, detailed trace of the agent's thoughts and actions is in the ADK web UI, which we'll set up later in this notebook.

And we'll cover more detailed workflows for logging and observability later in the course.

In [15]:
response = await runner.run_debug(" what is the temperature in Edmonton? ")


 ### Continue session: debug_session_id

User >  what is the temperature in Edmonton? 
helpful_assistant > The current temperature in Edmonton is 34°F (1°C). It feels like 28°F (-2°C) with a humidity of 61%.


# Section 3 - Web interface


In [16]:
!adk create sample-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY


Agent created in /Users/mario/Desktop/kaggle/Day-1/sample-agent:
- .env
- __init__.py
- agent.py

